In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

import pandas as pd
from scipy.spatial.distance import cdist
import seaborn as sns
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

from BEKML import BEMKL, poly_kernel, gauss_kernel, scoring

sns.set(style='ticks', context='talk')
np.set_printoptions(precision=4, linewidth=100)

Populating the interactive namespace from numpy and matplotlib


In [6]:
data = pd.read_csv('data/pima.csv', names=list(range(9)))
display(data.head())
data.describe()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


,0,1,2,3,4,5,6,7,8
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [8]:
X = data.iloc[:, :-1].copy()
y = data.iloc[:, -1].copy()
N, D = X.shape
y.loc[y == 0] = -1
display(X.shape, y.shape, y.mean())

(768, 8)

(768,)

-0.3020833333333333

In [9]:
kernels = [lambda A, B: gauss_kernel(A, B, 2**i) for i in range(-3, 7)]
kernels += [lambda A, B: poly_kernel(A, B, 0, i) for i in range(1, 4)]
kernels += [lambda A, B: gauss_kernel(A[:, j:j+1], B[:, j:j+1], 2**i) for i in range(-3, 7) for j in range(D)]
kernels += [lambda A, B: poly_kernel(A[:, j:j+1], B[:, j:j+1], 0, i) for i in range(1, 4) for j in range(D)]
sparse_model = BEMKL(kernels=kernels, alpha_lambda=1, beta_lambda=1, alpha_gamma=1, beta_gamma=1, alpha_omega=1e-11, beta_omega=1e+9)
normal_model = BEMKL(kernels=kernels, alpha_lambda=1, beta_lambda=1, alpha_gamma=1, beta_gamma=1, alpha_omega=1, beta_omega=1)
len(kernels)

117

In [10]:
sparse_pipeline = make_pipeline(Normalizer(), sparse_model)
normal_pipeline = make_pipeline(Normalizer(), normal_model)

In [12]:
scoring.iteration = 0
folds = RepeatedStratifiedKFold(n_splits=3, n_repeats=1)
cross_validate(sparse_pipeline, X, y, cv=folds, scoring=scoring)

0 - Non-0: 117. Total: 117. Ratio: 1.0
1 - Non-0: 117. Total: 117. Ratio: 1.0
2 - Non-0: 117. Total: 117. Ratio: 1.0


{'fit_time': array([ 10.324 ,  11.1101,  11.9295]),
 'score_time': array([ 0.6374,  0.7579,  0.5956]),
 'test_score': array([ 0.6732,  0.668 ,  0.7294]),
 'train_score': array([ 0.7123,  0.7168,  0.7076])}

In [13]:
scoring.iteration = 0
folds = RepeatedStratifiedKFold(n_splits=3, n_repeats=1)
cross_validate(normal_pipeline, X, y, cv=folds, scoring=scoring)

0 - Non-0: 117. Total: 117. Ratio: 1.0
1 - Non-0: 117. Total: 117. Ratio: 1.0
2 - Non-0: 117. Total: 117. Ratio: 1.0


{'fit_time': array([  9.7633,  10.0272,   9.9774]),
 'score_time': array([ 0.604 ,  0.5868,  0.5934]),
 'test_score': array([ 0.7315,  0.6797,  0.6588]),
 'train_score': array([ 0.7065,  0.7109,  0.7212])}

=====================

In [14]:
kernels = [lambda A, B: gauss_kernel(A, B, 2**i) for i in range(-3, 7)]
kernels += [lambda A, B: poly_kernel(A, B, 1, i) for i in range(1, 4)]
kernels += [lambda A, B: gauss_kernel(A[:, j:j+1], B[:, j:j+1], 2**i) for i in range(-3, 7) for j in range(D)]
kernels += [lambda A, B: poly_kernel(A[:, j:j+1], B[:, j:j+1], 1, i) for i in range(1, 4) for j in range(D)]
sparse_model = BEMKL(kernels=kernels, alpha_lambda=1, beta_lambda=1, alpha_gamma=1, beta_gamma=1, alpha_omega=1e-11, beta_omega=1e+9)
normal_model = BEMKL(kernels=kernels, alpha_lambda=1, beta_lambda=1, alpha_gamma=1, beta_gamma=1, alpha_omega=1, beta_omega=1)
len(kernels)

117

In [15]:
sparse_pipeline = make_pipeline(Normalizer(), sparse_model)
normal_pipeline = make_pipeline(Normalizer(), normal_model)

In [16]:
scoring.iteration = 0
folds = RepeatedStratifiedKFold(n_splits=3, n_repeats=1)
cross_validate(sparse_pipeline, X, y, cv=folds, scoring=scoring)

0 - Non-0: 117. Total: 117. Ratio: 1.0
1 - Non-0: 117. Total: 117. Ratio: 1.0
2 - Non-0: 117. Total: 117. Ratio: 1.0


{'fit_time': array([ 10.2817,  10.567 ,   9.8786]),
 'score_time': array([ 0.6036,  0.5902,  0.5888]),
 'test_score': array([ 0.7004,  0.707 ,  0.6745]),
 'train_score': array([ 0.7241,  0.7109,  0.7271])}

In [17]:
scoring.iteration = 0
folds = RepeatedStratifiedKFold(n_splits=3, n_repeats=1)
cross_validate(normal_pipeline, X, y, cv=folds, scoring=scoring)

0 - Non-0: 117. Total: 117. Ratio: 1.0
1 - Non-0: 117. Total: 117. Ratio: 1.0
2 - Non-0: 117. Total: 117. Ratio: 1.0


{'fit_time': array([  9.5895,  10.5979,  10.0612]),
 'score_time': array([ 0.6175,  0.5997,  0.6071]),
 'test_score': array([ 0.6848,  0.7188,  0.6902]),
 'train_score': array([ 0.7202,  0.7188,  0.7232])}